In [1]:
import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining


app_reviews = pd.read_csv("reviews.csv")

df_app_reviews = pd.DataFrame(app_reviews)

review_counter, column_counter = df_app_reviews.shape


print(review_counter)
print(column_counter)

94500
11


In [2]:
df_app_reviews.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ca5ea9e0-2388-4ea4-8c5c-05b54675fe87,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"aplikasinya sudah bagus, hanya saja metode pem...",5,49,11.3.0,2025-02-06 11:42:29,NaN,NaN,11.3.0
1,7179d985-57a8-44c1-9dec-e3ae68dfdb29,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Memasuki tahun 2025 BukaLapak jadi semakin mun...,1,30,11.3.0,2025-02-28 13:04:16,NaN,NaN,11.3.0
2,daba1033-9a55-4eaa-9878-a128f87ebb2e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Ketika transaksi pending, komplin ke cs sangat...",1,5,11.3.0,2025-02-27 08:05:14,NaN,NaN,11.3.0
3,78af8a64-91e7-47a0-b1c5-de20f94ffbf6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bukalapak makin gak jelas,pengajuan komplain p...",1,258,11.3.0,2025-01-14 21:53:33,NaN,NaN,11.3.0
4,8df46ba7-a6fa-4728-9e49-5e059d0f0e05,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,dengan ini saya memohon untuk pengembalian dan...,1,21,11.3.0,2025-02-21 15:14:50,NaN,NaN,11.3.0


In [3]:
df_app_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94500 entries, 0 to 94499
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              94500 non-null  object
 1   userName              94500 non-null  object
 2   userImage             94500 non-null  object
 3   content               94500 non-null  object
 4   score                 94500 non-null  int64 
 5   thumbsUpCount         94500 non-null  int64 
 6   reviewCreatedVersion  67907 non-null  object
 7   at                    94500 non-null  object
 8   replyContent          26551 non-null  object
 9   repliedAt             26551 non-null  object
 10  appVersion            67907 non-null  object
dtypes: int64(2), object(9)
memory usage: 7.9+ MB


In [4]:
clean_df_app_reviews = df_app_reviews.dropna()
clean_df_app_reviews = clean_df_app_reviews.drop_duplicates()
clean_df_app_reviews = clean_df_app_reviews.drop(columns=["reviewId","userName","userImage","score","thumbsUpCount","reviewCreatedVersion","at","replyContent","repliedAt","appVersion"])

clean_review_counter, clean_column_counter = clean_df_app_reviews.shape

clean_df_app_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20525 entries, 11 to 94491
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  20525 non-null  object
dtypes: object(1)
memory usage: 320.7+ KB


Processing Text

In [5]:
%pip install sastrawi nltk wordcloud

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt_tab')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
 
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
 
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text
 
def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text
 
def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text
 
def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text
 
def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    # Memecah teks menjadi daftar kata
    words = text.split()
 
    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]
 
    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)
 
    return stemmed_text
 
def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

  Using cached Sastrawi-1.0.1-py2.py3-none-any.whl.metadata (909 bytes)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached wordcloud-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pillow-11.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached matplotlib-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata 

[nltk_data] Downloading package punkt_tab to /home/gandhi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/gandhi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal","gua": "saya","bod*h":"bodoh","henti""":"henti-henti","gada":"tidak ada"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [7]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df_app_reviews['text_clean'] = clean_df_app_reviews['content'].apply(cleaningText)
 
# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df_app_reviews['text_casefoldingText'] = clean_df_app_reviews['text_clean'].apply(casefoldingText)
 
# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df_app_reviews['text_slangwords'] = clean_df_app_reviews['text_casefoldingText'].apply(fix_slangwords)
 
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df_app_reviews['text_tokenizingText'] = clean_df_app_reviews['text_slangwords'].apply(tokenizingText)
 
# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df_app_reviews['text_stopword'] = clean_df_app_reviews['text_tokenizingText'].apply(filteringText)
 
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df_app_reviews['text_akhir'] = clean_df_app_reviews['text_stopword'].apply(toSentence)

In [8]:
clean_df_app_reviews

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
11,Saya top up dana Sudah saya transfer tapi di a...,Saya top up dana Sudah saya transfer tapi di a...,saya top up dana sudah saya transfer tapi di a...,saya top up dana sudah saya transfer tapi di a...,"[saya, top, up, dana, sudah, saya, transfer, t...","[top, up, dana, transfer, aplikasi, masuk, moh...",top up dana transfer aplikasi masuk mohon diba...
24,"Nama kami dulu besar di sini, performa lapak s...",Nama kami dulu besar di sini performa lapak sa...,nama kami dulu besar di sini performa lapak sa...,nama kami dulu besar di sini performa lapak sa...,"[nama, kami, dulu, besar, di, sini, performa, ...","[nama, performa, lapak, bagus, sayangkan, semn...",nama performa lapak bagus sayangkan semnjak vr...
42,mau komplen kemana gak jelas. cuman diarahin k...,mau komplen kemana gak jelas cuman diarahin ke...,mau komplen kemana gak jelas cuman diarahin ke...,mau komplen kemana gak jelas cuman diarahin ke...,"[mau, komplen, kemana, gak, jelas, cuman, diar...","[komplen, kemana, cuman, diarahin, faq, update...",komplen kemana cuman diarahin faq update bukab...
63,Setelah update malah jadi suka error padahal s...,Setelah update malah jadi suka error padahal s...,setelah update malah jadi suka error padahal s...,setelah update malah jadi suka error padahal s...,"[setelah, update, malah, jadi, suka, error, pa...","[update, suka, error, cepat, suka, errorbarusa...",update suka error cepat suka errorbarusan turu...
97,Sudah bertahun² gak pernah dikasih vocer poton...,Sudah bertahun² gak pernah dikasih vocer poton...,sudah bertahun² gak pernah dikasih vocer poton...,sudah bertahun² gak pernah dikasih vocer poton...,"[sudah, bertahun², gak, pernah, dikasih, vocer...","[bertahun², dikasih, vocer, potongan, harga, g...",bertahun² dikasih vocer potongan harga gratis ...
...,...,...,...,...,...,...,...
94481,Masih kalah dengan aplikasi Tokopedia dari seg...,Masih kalah dengan aplikasi Tokopedia dari seg...,masih kalah dengan aplikasi tokopedia dari seg...,masih kalah dengan aplikasi tokopedia dari seg...,"[masih, kalah, dengan, aplikasi, tokopedia, da...","[kalah, aplikasi, tokopedia, fitur]",kalah aplikasi tokopedia fitur
94484,"fitur chat k pelapak nya ga konek, cuma contre...",fitur chat k pelapak nya ga konek cuma contren...,fitur chat k pelapak nya ga konek cuma contren...,fitur chat k pelapak nya ga konek cuma contren...,"[fitur, chat, k, pelapak, nya, ga, konek, cuma...","[fitur, chat, k, pelapak, konek, contreng, dah...",fitur chat k pelapak konek contreng dah berhar...
94488,"Parah nih Bukalapak, masa bukadompet saya di b...",Parah nih Bukalapak masa bukadompet saya di be...,parah nih bukalapak masa bukadompet saya di be...,parah nih bukalapak masa bukadompet saya di be...,"[parah, nih, bukalapak, masa, bukadompet, saya...","[parah, nih, bukalapak, bukadompet, bekukan, a...",parah nih bukalapak bukadompet bekukan ampun s...
94489,Kalo flash deal tentang peralatan sepeda dong ...,Kalo flash deal tentang peralatan sepeda dong ...,kalo flash deal tentang peralatan sepeda dong ...,kalo flash deal tentang peralatan sepeda dong ...,"[kalo, flash, deal, tentang, peralatan, sepeda...","[kalo, flash, deal, peralatan, sepeda, shifter]",kalo flash deal peralatan sepeda shifter


In [9]:
%pip install requests
import csv
import requests
from io import StringIO
 
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")
 
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (146 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Fungsi untuk menentukan polaritas sentimen dari tweet
 
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
 
    score = 0
    # Inisialisasi skor sentimen ke 0
 
    for word in text:
        # Mengulangi setiap kata dalam teks
        
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen
        elif (word in lexicon_negative):
            score = score + lexicon_negative[word]
        else:
            score = 0
 
    polarity=''
    # Inisialisasi variabel polaritas
 
    if (score > 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [17]:
results = clean_df_app_reviews['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df_app_reviews['polarity_score'] = results[0]
clean_df_app_reviews['polarity'] = results[1]
print(clean_df_app_reviews['polarity'].value_counts())

polarity
neutral     13153
positive     4447
negative     2925
Name: count, dtype: int64


In [32]:
%pip install scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df_app_reviews['text_akhir']
y = clean_df_app_reviews['polarity']
 
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)
 
# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
features_df
# Menampilkan hasil ekstraksi fitur
features_df.info()
 
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2,random_state=42)

Note: you may need to restart the kernel to use updated packages.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20525 entries, 0 to 20524
Columns: 200 entries, admin to yg
dtypes: float64(200)
memory usage: 31.3 MB


In [33]:
from sklearn.metrics import accuracy_score # type: ignore
from sklearn.tree import DecisionTreeClassifier # type: ignore
 
# Membuat objek model Decision Tree
decision_tree = DecisionTreeClassifier()
 
# Melatih model Decision Tree pada data pelatihan
decision_tree.fit(X_train.toarray(), y_train)
 
# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())
 
# Evaluasi akurasi model Decision Tree
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)
 
# Menampilkan akurasi
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)


Decision Tree - accuracy_train: 0.9862971985383678
Decision Tree - accuracy_test: 0.5198538367844092


In [36]:
from sklearn.ensemble import RandomForestClassifier # type: ignore
 
# Membuat objek model Random Forest
random_forest = RandomForestClassifier()
 
# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)
 
# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())
 
# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)
 
# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9862971985383678
Random Forest - accuracy_test: 0.6531059683313033
